# Entrenamiento de YOLOv8 en Google Colab (Dataset Pre-descomprimido)

Este notebook está adaptado para entrenar un modelo YOLOv8, asumiendo que el dataset ya está descomprimido en Google Drive. Guarda los resultados en Google Drive y tiene capacidad de reanudar el entrenamiento si la sesión se interrumpe.

### 1. Montar Google Drive

Conecta el notebook a tu Google Drive. Aquí es donde leeremos el dataset y guardaremos los resultados y checkpoints.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2. Instalar Librerías

Instalamos `ultralytics` para YOLOv8.

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.1 MB/s eta 0:00:00


### 3. Configuración del Entrenamiento

**ACCIÓN REQUERIDA:** Ajusta las rutas y parámetros en la siguiente celda según tu configuración.

In [ ]:
import os
import tarfile
import zipfile
from ultralytics import YOLO

# --- PARÁMETROS DE CONFIGURACIÓN ---

# Asegurarse de que CUDA_VISIBLE_DEVICES esté configurado para usar la primera GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Ruta base en tu Google Drive donde tienes el proyecto
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision' # Asegúrate de que esta ruta sea correcta

# --- Configuración del Dataset (comprimido) ---
# Nombre del archivo del dataset comprimido en Drive
COMPRESSED_DATASET_NAME = 'dataset_v2_yolov8_obb.tar.gz' # o 'dataset_v2_yolov8_obb.zip'
COMPRESSED_DATASET_PATH = os.path.join(DRIVE_PROJECT_PATH, COMPRESSED_DATASET_NAME)

# Ruta donde se descomprimirá el dataset en el ENTORNO LOCAL de Colab (mucho más rápido)
# El nombre de la carpeta debe coincidir con el que se crea al descomprimir
DATASET_BASE_PATH = '/content/dataset_v2_yolov8_obb'

# Ruta al archivo YAML del dataset (apuntando a la carpeta local)
DATASET_YAML_PATH = os.path.join(DATASET_BASE_PATH, 'data.yaml')


# --- Lógica de Descompresión ---
# Comprobar si el dataset ya ha sido descomprimido en esta sesión de Colab
if not os.path.exists(DATASET_BASE_PATH):
    print(f"Dataset no encontrado en '{DATASET_BASE_PATH}'.")
    print(f"Iniciando descompresión de '{COMPRESSED_DATASET_PATH}'...")

    if not os.path.exists(COMPRESSED_DATASET_PATH):
        raise FileNotFoundError(f"El archivo comprimido no se encontró en {COMPRESSED_DATASET_PATH}. Verifica la ruta y el nombre.")

    # --- Elige UNA de las siguientes opciones según tu tipo de archivo ---

    # Opción 1: Para archivos .tar.gz (Recomendado)
    with tarfile.open(COMPRESSED_DATASET_PATH, 'r:gz') as tar_ref:
        tar_ref.extractall('/content/') # Extraer en la raíz del entorno local

    # # Opción 2: Para archivos .zip (Descomenta esta sección si usas .zip)
    # with zipfile.ZipFile(COMPRESSED_DATASET_PATH, 'r') as zip_ref:
    #     zip_ref.extractall('/content/') # Extraer en la raíz del entorno local

    print("Descompresión completada.")
else:
    print(f"Dataset ya se encuentra descomprimido en '{DATASET_BASE_PATH}'.")


# --- Configuración del Modelo y Entrenamiento ---
MODEL_NAME = os.path.join(DRIVE_PROJECT_PATH, 'models', 'yolov8m.pt')
EPOCHS = 80
RUN_NAME = 'yolov8m_obb_colab_b32_w8_img640_ep80_run_3'

# Parámetros de rendimiento para Colab
# No termina entrenamiento con:
#   - BATCH_SIZE = 32
#   - WORKERS = 8
BATCH_SIZE = 16
IMG_SIZE = 640
WORKERS = 4
patience = 20
cache = True
optimizer = 'AdamW'
cos_lr = True

# --- FIN DE LA CONFIGURACIÓN ---

# Asegurarse de que la carpeta de resultados exista en Drive
os.makedirs(os.path.join(DRIVE_PROJECT_PATH, 'training_results'), exist_ok=True)

# --- Verificación del Dataset y Modelo Base ---
if not os.path.exists(DATASET_YAML_PATH):
    print(f"ERROR: El archivo data.yaml no se encontró en {DATASET_YAML_PATH}. La descompresión pudo haber fallado o la ruta es incorrecta.")
    raise FileNotFoundError(f"Dataset YAML not found at {DATASET_YAML_PATH}")

if not os.path.exists(MODEL_NAME):
    print(f"ADVERTENCIA: El modelo base {MODEL_NAME} no se encontró. Ultralytics lo descargará automáticamente.")
    MODEL_NAME = 'yolov8m.pt'

print(f"Dataset listo para usar en: {DATASET_BASE_PATH}")
print(f"Archivo YAML: {DATASET_YAML_PATH}")
print(f"Modelo base: {MODEL_NAME}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Dataset no encontrado en '/content/dataset_v2_yolov8_obb'.
Iniciando descompresión de '/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision/dataset_v2_yolov8_obb.tar.gz'...


/tmp/ipython-input-676549626.py:40: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar_ref.extractall('/content/') # Extraer en la raíz del entorno local


Descompresión completada.
Dataset listo para usar en: /content/dataset_v2_yolov8_obb
Archivo YAML: /content/dataset_v2_yolov8_obb/data.yaml
Modelo base: /content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision/models/yolov8m.pt


### 4. Entrenar el Modelo

Esta celda inicia o reanuda el entrenamiento. Gracias a `resume=True`, si el script se detiene, puedes volver a ejecutar esta celda y continuará donde lo dejó.

In [ ]:
from ultralytics import YOLO

# Cargar el modelo base (YOLO descargará 'yolov8m.pt' si no existe)
model = YOLO(MODEL_NAME)

# Iniciar el entrenamiento (versión corregida)
results = model.train(
    # --- Rutas y Datos ---
    data=DATASET_YAML_PATH,
    project=os.path.join(DRIVE_PROJECT_PATH, 'training_results'),
    name=RUN_NAME,

    # --- Parámetros de Entrenamiento ---
    epochs=EPOCHS,
    patience=patience,

    # --- Parámetros de Rendimiento y Optimización ---
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    workers=WORKERS,
    cache=cache,
    optimizer=optimizer,
    cos_lr=cos_lr,
    weight_decay=0.0005, # <-- ESTE SÍ es correcto y recomendable para prevenir overfitting

    # --- Opcionales ---
    save_json=True,
    # save_hybrid=True, # <-- Eliminado por estar obsoleto
    plots=True,
    device='0'
)

print('--- ENTRENAMIENTO FINALIZADO ---')
print(f'Resultados guardados en: {results.save_dir}')

Ultralytics 8.3.192 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset_v2_yolov8_obb/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision/models/yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m_obb_colab_b32_w8_imgsz640_80_epochs_run_3, nbs=64

### 5. Evaluar el Modelo Final

Una vez finalizado el entrenamiento completo, puedes ejecutar esta celda para ver las métricas de validación del mejor modelo.

In [ ]:
# Cargar el mejor modelo guardado durante el entrenamiento
best_model_path = os.path.join(DRIVE_PROJECT_PATH, 'training_results', RUN_NAME, 'weights', 'best.pt')
model = YOLO(MODEL_NAME)

# Evaluar en el conjunto de validación
metrics = model.val()

print('--- MÉTRICAS DE VALIDACIÓN ---')
print(f'mAP50-95: {metrics.box.map:.4f}')
print(f'   mAP50: {metrics.box.map50:.4f}')
print(f'   mAP75: {metrics.box.map75:.4f}')